# **WAVE MOBILE MONEY**

The project is about a money transfer business called Wave and sql queries were used to answer questions on it.

In [ ]:
# Importing essential resources to the project.

In [4]:
!pip install ipython-sql
%load_ext sql
import sqlite3

     -------------------------------------- 41.2/41.2 kB 197.1 kB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 892.6 kB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.39
    Uninstalling SQLAlchemy-1.4.39:
      Successfully uninstalled SQLAlchemy-1.4.39


In [14]:
import pandas as pd

In [ ]:
# Establishing a connection with the database known as Wave as well as the cursor to execute.

In [21]:
conn = sqlite3.connect('Wave.db')
cur = conn.cursor()


In [22]:
%sql sqlite:///Wave.db

In [ ]:
# The data are in several csv files which are read and stored in sql format

In [23]:
df = pd.read_csv(R"C:\Users\wolak\Downloads\DatabaseProjectFiles220801081355-220815-093656\agents.csv")
df.to_sql("AGENTS", conn, if_exists = 'replace',index=False, method = 'multi')

10

In [25]:
df=pd.read_csv(R"C:\Users\wolak\Downloads\DatabaseProjectFiles220801081355-220815-093656\agent_transactions.csv")

df.to_sql("AGENT_TRANSACTIONS", conn, if_exists = "replace",index=False, method = "multi")

df=pd.read_csv(R"C:\Users\wolak\Downloads\DatabaseProjectFiles220801081355-220815-093656\transfers.csv")

df.to_sql("TRANSFERS", conn, if_exists = "replace",index=False, method = "multi")

df=pd.read_csv(R"C:\Users\wolak\Downloads\DatabaseProjectFiles220801081355-220815-093656\wallets.csv")

df.to_sql("WALLETS", conn, if_exists = "replace",index=False, method = "multi")

df=pd.read_csv(R"C:\Users\wolak\Downloads\DatabaseProjectFiles220801081355-220815-093656\users.csv")

df.to_sql("USERS", conn, if_exists = "replace",index=False, method = "multi")

10

In [ ]:
# Double checking to see if all csv load and sql saves were succesful using sqlite_master.

In [26]:
%sql SELECT name FROM sqlite_master WHERE type='table';

   sqlite:///Shipment.db
 * sqlite:///Wave.db
Done.


name
AGENTS
AGENT_TRANSACTIONS
TRANSFERS
WALLETS
USERS


## 1. How many users does Wave have?

In [27]:
%sql SELECT COUNT (*) AS No_users FROM users;

   sqlite:///Shipment.db
 * sqlite:///Wave.db
Done.


No_users
10


## 2. How many transfers have been sent in the currency CFA?

In [31]:
%%sql SELECT kind, COUNT(send_amount_currency) as Transfers_in_CFA
FROM transfers WHERE send_amount_currency = 'CFA'
GROUP BY kind;

   sqlite:///Shipment.db
 * sqlite:///Wave.db
Done.


kind,Transfers_in_CFA
Merchant,2
P2P,4


## 3. How many different users have sent a transfer in CFA?

A transfer in Wave is when a user (denoted by their user_id in the u_id column) sends money from their Wave account (often to another Wave user but the recipient might also be a mobile money user that hasn't signed up for an account yet, or the sender of the transfer could be making a bill payment, or a few other possibilities; the kind column stores which possibility.

In [33]:
%%sql SELECT user_id, kind, COUNT(send_amount_currency) as Transfers_in_CFA
FROM transfers WHERE send_amount_currency = 'CFA'
GROUP BY user_id, kind;

   sqlite:///Shipment.db
 * sqlite:///Wave.db
Done.


user_id,kind,Transfers_in_CFA
1,Merchant,1
4,P2P,1
5,P2P,1
6,P2P,1
7,P2P,1
8,Merchant,1


## 4. How many agent transactions did we encounter in the months of 2022? (broken down by months)

An agent transaction is when someone deposits or withdraws money from a Wave agent and an agent is a person or business that is contracted to facilitate transactions for users. The most important of these are cash-in and cash-out (ie. loading value into the mobile money system and then converting it back out again).

In [37]:
%%sql select 
COUNT(atx_id) as no_of_atx,
SUBSTRING (when_created, 4,2) as month
FROM agent_transactions
GROUP BY month
ORDER BY month asc;

   sqlite:///Shipment.db
 * sqlite:///Wave.db
Done.


no_of_atx,month
4,01
8,02
5,03
5,04
2,05
3,06
6,07
2,08
2,09
1,10


## 5. Over the course of the first half of 2022, how many wave agents were "net depositors" vs "net withdrawers"?

A net depositor is an agent who received more deposit volume than withdrawal volume over some given time frame( and vice versa for a net withdrawer). Whether an agent transaction is a deposit or a withdrawal is determined by the sign of its amount: a negative amount is a deposit, and a positive amount is a withdrawal; zero amounts aren't allowed.

In [53]:
%%sql select DISTINCT
sum(case when amount > 0 then 1 else 0 end) as Net_withdrawer,
sum(case when amount < 0 then 1 else 0 end) as Number_Of_Net_Depositor
from agent_transactions atr
join agents ag ON
atr.agent_id = ag.agent_id
where atr.when_created between '01/01/2022 00:00' and '30/06/2022 00:00';

   sqlite:///Shipment.db
 * sqlite:///Wave.db
Done.


Net_withdrawer,Number_Of_Net_Depositor
30,9


## 6. Find the volume of agenttransactions created in the first half of 2022, grouped by city. 

The city was determined where the agent transaction took place from the agent's city field.

In [42]:
%%sql 
SELECT DISTINCT ag.city, count(atr.atx_id) as volume
FROM agent_transactions atr
JOIN agents ag ON
atr.agent_id = ag.agent_id
WHERE atr.when_created BETWEEN '01/01/2022 00:00' AND '30/06/2022 12:00'
GROUP BY ag.city;


   sqlite:///Shipment.db
 * sqlite:///Wave.db
Done.


city,volume
Abidjan,8
Abuja,5
Accra,8
Dakar,7
Kumasi,4
Lagos,4
Takoradi,3


## 7. Separate the volume by country as well(columns should now have country, city and volume).

In [41]:
%%sql 
SELECT DISTINCT ag.country, ag.city, count(atr.atx_id) as volume
FROM agent_transactions atr
JOIN agents ag ON
atr.agent_id = ag.agent_id
WHERE atr.when_created BETWEEN '01/01/2022 00:00' AND '30/06/2022 12:00'
GROUP BY ag.city, ag.country;


   sqlite:///Shipment.db
 * sqlite:///Wave.db
Done.


country,city,volume
Ivory Coast,Abidjan,8
Nigeria,Abuja,5
Ghana,Accra,8
Senegal,Dakar,7
Ghana,Kumasi,4
Nigeria,Lagos,4
Ghana,Takoradi,3


## 8. Build a "send volume by country and kind" table: find the total volume of transfers (by send_amount_scalar) sent in the first half of 2022, grouped by country and transfer kind.

There are a few different kinds of Wave transfers between a Wave user and a mobile number, or a payment from the Wave user toa merchant. The country a transfer was sent from isn't stored on the transfers table itself, but rather on the wallet's table's ledger_location field (and each transfer has a source_wallet_id).

In [48]:
%%sql 
SELECT DISTINCT wt.ledger_location, tr.kind, count(send_amount_scalar) as volume_of_transfers
FROM transfers tr
JOIN wallets wt ON wt.wallet_id = tr.source_wallet_id
WHERE tr.when_created BETWEEN '01/01/2022 00:00' AND '30/06/2022 23:59'
GROUP BY wt.ledger_location,tr.kind ;


   sqlite:///Shipment.db
 * sqlite:///Wave.db
Done.


ledger_location,kind,volume_of_transfers
Germany,Merchant,4
Germany,P2P,2
Ghana,Merchant,6
Ghana,P2P,2
Ivory Coast,P2P,6
USA,Merchant,3
USA,P2P,1


## 9. Add columns for the unique senders (still broken down by country and kind)

In [47]:
%%sql SELECT DISTINCT(tr.user_id)as senders,wt.ledger_location, tr.kind, count(tr.send_amount_scalar) as vol_of_transfers
FROM transfers tr
JOIN wallets wt ON wt.wallet_id = tr.source_wallet_id
WHERE tr.when_created BETWEEN '01/01/2022 00:00' AND '30/06/2022 23:59'
GROUP BY tr.user_id, wt.ledger_location,tr.kind;

   sqlite:///Shipment.db
 * sqlite:///Wave.db
Done.


senders,ledger_location,kind,vol_of_transfers
1,Ghana,Merchant,3
2,Ghana,P2P,1
2,USA,Merchant,2
3,Germany,Merchant,1
3,Ivory Coast,P2P,1
3,USA,Merchant,1
4,Ghana,Merchant,1
4,Ivory Coast,P2P,1
5,Germany,P2P,1
5,Ivory Coast,P2P,1


## 10. Which wallets sent more than 1,000,000 CFA in transfers (as identified by the source_wallet_id column on the transfers table) and how much did they spend?

In [50]:
%%sql
SELECT wt.wallet_id, tr.send_amount_scalar, tr.when_created
FROM transfers tr
JOIN wallets wt ON wt.wallet_id = tr.source_wallet_id
WHERE tr.send_amount_scalar > 1000000;

   sqlite:///Shipment.db
 * sqlite:///Wave.db
Done.


wallet_id,send_amount_scalar,when_created
5,2000000,06/02/2022 00:00
7,1400000,10/05/2022 00:00
9,12150000,21/05/2022 00:00


In [ ]:
#Closing the connection

In [52]:
conn.close()